In [1267]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [1268]:
df = pd.read_csv('datasets\caracteres-ruido.csv', header=None)

ruido = pd.read_csv('datasets\caracteres-ruido20.csv', header=None)

df

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,-1,-1,1,1,-1,-1,-1,-1,-1,1,...,1,-1,1,1,-1,-1,-1,-1,-1,-1
1,1,1,1,1,1,1,-1,-1,-1,-1,...,1,1,-1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,1,1,1,1,1,1,-1,...,1,1,-1,-1,-1,1,-1,-1,-1,-1
3,1,1,1,1,1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,-1,1,-1,-1,-1
4,1,1,-1,1,1,1,1,-1,1,-1,...,1,1,1,-1,-1,-1,-1,1,-1,-1
5,-1,-1,-1,1,1,-1,1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,1,-1
6,1,-1,1,-1,1,1,1,-1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
7,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,...,1,-1,1,1,-1,-1,-1,-1,-1,-1
8,1,1,1,1,1,1,-1,1,-1,-1,...,1,1,-1,-1,1,-1,-1,-1,-1,-1
9,-1,-1,1,1,1,-1,-1,-1,1,-1,...,1,-1,-1,-1,-1,1,-1,-1,-1,-1


# Hiperparametros

## funções

In [1269]:
Activations = {
    'tanh' : (lambda x: np.tanh(x))
    }
D_Activations = {
    'tanh' : (lambda x: 1-x**2)
    }
Errors = {
    'SR' : (lambda x,y: (x-y)**2)
    }
D_Errors = {
    'SR' : (lambda x,y: -2*(x-y))
    }

In [1270]:
L0 = 63
L1 = 20
L2 = 7

learning_rate = 0.05
max_epochs = 500

activation_func = Activations['tanh']
activation_deriv = D_Activations['tanh']

error_func = Errors['SR']
error_deriv = D_Errors['SR']

max_lim = 1
min_lim = -1

classes_number = 7

# Parametros

In [1271]:
weights_L1 = [[]]
bias_L1 = []
weights_L2 = [[]]
bias_L2 = []

# implementação do algoritmo

## Inicialização dos pesos e viéses

In [1272]:
def start_weights(i, j):
    weights = []
    for _ in range(j):
        temp = []
        for _ in range(i):
            temp.append(random.uniform(min_lim, max_lim)+.1)
        weights.append(temp)
    return np.array(weights)

In [1273]:
def start_bias(j):
    bias = []
    for _ in range(j):
        bias.append(random.uniform(min_lim, max_lim)+.1)
    return np.array(bias)

## Feed forward

In [1274]:
def feed_forward(x):
    temp = []

    output_L1 = []
    output_L2 = []

    for i,w in enumerate(weights_L1):
        temp = [X*W for (X, W) in zip(x, w)]
        temp = np.sum(temp) + bias_L1[i]
        output_L1.append(activation_func(temp))

    temp = []

    for i,w in enumerate(weights_L2):
        temp = [X*W for (X, W) in zip(output_L1, w)]
        temp = np.sum(temp) + bias_L2[i]
        output_L2.append(activation_func(temp))

    return np.array(output_L1), np.array(output_L2)
    

## calculo do erro total

In [1275]:
def error_calculation(y, output_L2):
    total_error = 0
    for i in range(L2):
        SR = error_func(y[i], output_L2[i])
        total_error += SR
    return total_error

## back propagation

In [1276]:
def back_propagation(x, y, output_L1, output_L2):
    # gradient descent L2
    delta_L2 = error_deriv(y, output_L2) * activation_deriv(output_L2)

    # updating weights and biases L2
    for i in range(L2):
        for j in range(L1):
            weights_L2[i][j] -= learning_rate * delta_L2[i] * output_L1[j]
            bias_L2[i] -= learning_rate * delta_L2[i]

    # gradient descent L1
    delta_L1 = []
    for v in weights_L2.T:
        delta_L1.append(np.sum(v*delta_L2))

    delta_L1 = delta_L1 * activation_deriv(output_L1)
    delta_L1 = np.array(delta_L1) 

    # updating weights adn biases L1
    for i in range(L1):
        for j in range(L0):
            weights_L1[i][j] -= (learning_rate * (delta_L1[i] * x[j]))
            bias_L1[i] -= (learning_rate * delta_L1[i])

    

## algoritmo de treinamento do modelo

In [1277]:
def fit(X, Y):
    epoch = 0

    while epoch < max_epochs:
        for i in range(len(X.index)):
            x = np.array(X.iloc[i].values)
            y = np.array(Y.iloc[i].values)
            
            # feed forward
            output_L1, output_L2 = feed_forward(x)

           

            # error calculation
            total_error = error_calculation(y, output_L2)

            # back propagation
            back_propagation(x, y, output_L1, output_L2)
        
        total_error = (total_error / len(X.index))

        epoch += 1

## algoritmo de teste

In [1278]:
def output_treatment(output):
    #print(output)
    temp = np.empty(len(output))
    for i, o in enumerate(output):
        if o > 0:
            temp[i] = 1
        else:
            temp[i] = 0 #-1
    
    return temp



In [1279]:
def predict(X):
    preds = []
    

    for i in range(len(X.index)):
        x = np.array(X.iloc[i].values)

        output_L1, output_L2 = feed_forward(x)

        output_L2 = output_treatment(output_L2)

        preds.append(output_L2)

    preds = pd.DataFrame(preds)
    preds = preds.astype('int64')

    return preds

# Uso da IA

## Separação de treino e teste

In [1280]:
X = df.iloc[:,0:63]
X_ruido = ruido.iloc[:,0:63]
y_ruido = ruido.iloc[:,63:70]

X.head()

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,-1,-1,1,1,-1,-1,-1,-1,-1,1,...,-1,1,-1,1,-1,1,-1,1,-1,1
1,1,1,1,1,1,1,-1,-1,-1,-1,...,-1,-1,1,1,1,1,1,1,1,-1
2,-1,-1,1,1,1,1,1,1,1,-1,...,1,-1,1,-1,-1,1,1,1,1,-1
3,1,1,1,1,1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,1,1,1,-1,-1
4,1,1,-1,1,1,1,1,-1,1,-1,...,-1,-1,1,1,1,1,1,1,1,1


In [1281]:
y = df.iloc[:,63:70]

y.head()

,63,64,65,66,67,68,69
0,1,-1,-1,-1,-1,-1,-1
1,-1,1,-1,-1,-1,-1,-1
2,-1,-1,1,-1,-1,-1,-1
3,-1,-1,-1,1,-1,-1,-1
4,-1,-1,-1,-1,1,-1,-1


In [1282]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=7, stratify=y)

In [1283]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
2,-1,-1,1,1,1,1,1,1,1,-1,...,1,-1,1,-1,-1,1,1,1,1,-1
5,-1,-1,-1,1,1,-1,1,-1,-1,-1,...,1,1,-1,-1,-1,1,1,1,-1,-1
20,-1,1,1,-1,-1,1,-1,-1,1,-1,...,-1,-1,-1,-1,1,1,-1,-1,1,1
4,1,1,-1,1,1,1,1,-1,1,-1,...,-1,-1,1,1,1,1,1,1,1,1
12,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,...,-1,1,-1,1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,1,-1,-1,-1,-1,...,-1,-1,1,1,1,1,1,1,1,-1
16,-1,-1,1,1,1,-1,1,-1,1,-1,...,-1,1,-1,-1,-1,-1,1,1,-1,-1
11,-1,1,1,1,1,1,1,1,-1,-1,...,-1,-1,-1,-1,1,1,1,1,1,1
6,1,-1,1,-1,1,1,1,-1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,1,1
8,1,1,1,1,1,1,-1,1,-1,-1,...,-1,-1,-1,1,1,1,1,1,1,-1


In [1284]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
15,1,1,1,1,1,1,-1,1,1,-1,...,-1,-1,1,1,1,1,1,1,1,-1
19,-1,-1,-1,-1,1,1,1,-1,-1,-1,...,-1,-1,-1,-1,-1,1,1,1,-1,-1
13,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,1,1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,1,1,1,-1,-1
3,1,1,1,1,1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,1,1,1,-1,-1
18,1,1,1,1,1,1,1,-1,1,-1,...,-1,-1,1,-1,1,1,-1,1,1,1
7,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,1,-1,-1,-1,1,-1,1


In [1285]:
y_train

,63,64,65,66,67,68,69
2,-1,-1,1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,1,-1
20,-1,-1,-1,-1,-1,-1,1
4,-1,-1,-1,-1,1,-1,-1
12,-1,-1,-1,-1,-1,1,-1
1,-1,1,-1,-1,-1,-1,-1
16,-1,-1,1,-1,-1,-1,-1
11,-1,-1,-1,-1,1,-1,-1
6,-1,-1,-1,-1,-1,-1,1
8,-1,1,-1,-1,-1,-1,-1


In [1286]:
y_test

,63,64,65,66,67,68,69
15,-1,1,-1,-1,-1,-1,-1
19,-1,-1,-1,-1,-1,1,-1
13,-1,-1,-1,-1,-1,-1,1
9,-1,-1,1,-1,-1,-1,-1
3,-1,-1,-1,1,-1,-1,-1
18,-1,-1,-1,-1,1,-1,-1
7,1,-1,-1,-1,-1,-1,-1


## treinando

In [1287]:
weights_L1 = start_weights(L0, L1)
bias_L1 = start_bias(L1)
weights_L2 = start_weights(L1, L2)
bias_L2 = start_bias(L2)

fit(X, y)

## testando

In [1288]:
y_test = y_test.replace(-1, 0)
y_test.head(7)

,63,64,65,66,67,68,69
15,0,1,0,0,0,0,0
19,0,0,0,0,0,1,0
13,0,0,0,0,0,0,1
9,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0
18,0,0,0,0,1,0,0
7,1,0,0,0,0,0,0


In [1289]:
y_ruido = y_ruido.replace(-1, 0)
y_ruido.head(21)

,63,64,65,66,67,68,69
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0
5,0,0,0,0,0,1,0
6,0,0,0,0,0,0,1
7,1,0,0,0,0,0,0
8,0,1,0,0,0,0,0
9,0,0,1,0,0,0,0


In [1290]:
preds = predict(X_ruido)
preds.style.background_gradient(cmap ='viridis')
preds

,0,1,2,3,4,5,6
0,1,0,0,0,1,0,0
1,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0
5,0,0,0,0,0,1,0
6,0,0,0,0,0,0,1
7,1,0,0,0,1,0,0
8,0,1,0,0,0,0,0
9,0,0,1,0,0,0,0


In [1291]:
y_ruido

,63,64,65,66,67,68,69
0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0
5,0,0,0,0,0,1,0
6,0,0,0,0,0,0,1
7,1,0,0,0,0,0,0
8,0,1,0,0,0,0,0
9,0,0,1,0,0,0,0
